In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Call Api on File Upload

Attributes
----------
subcriptions
events
notifications
luminesce
"""

toggle_code("Toggle Docstring")

# Call API on File Upload

This notebook will detail how to create a process in which actions are triggered on the upload of a new file.

For this example, we will manually drop an auto-load-transactions.xlsx file into drive, and this will trigger an api call which will run a Luminesce query to create a portfolio, create instruments and upsert the transactions into that portfolio. This is a very simple example, but it could be extended further by triggering a job which could contain references to any number of apis.

The steps are as follows:
1. Authorise LUSID account
2. Create subscription to FileCreated event
3. Create webhook notification to be triggered by subscription
    - This notification will call an API
4. Trigger the event

### Setup Lusid

Import modules and define our api factories.

In [4]:
import os
import json

# import lusid specific packages
import lusid.models as models
from fbnsdkutilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid_notifications as ln

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")
api_url = os.getenv("FBN_LUSID_API_URL")
domain = api_url[:-3]

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

notifications_factory = ApiClientFactory(
    api_url=f"https://{domain}notifications",
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    sdk=ln,
)

In [ ]:
# notifications APIs
subscriptions_api = notifications_factory.build(ln.api.SubscriptionsApi)
notifications_api = notifications_factory.build(ln.api.NotificationsApi)
events_api = notifications_factory.build(ln.api.EventsApi)
event_types_api = notifications_factory.build(ln.api.EventTypesApi)

### 1. Authorise LUSID account

To authorise your LUSID account, follow the instruction at [KA-01735](https://support.lusid.com/knowledgebase/article/KA-01735/)


### 2. Create a subscription

Here we are creating a subscription to a `FileCreated` event, with a matching filter set to look for our `auto-load-transactions.xlsx` file.

In [5]:
# create subscription to file creation event
def create_file_subscription(
        scope: str,
        code: str,
        name: str,
        matching_filter: str,
        description: str = None
):
    try:
        resp = subscriptions_api.create_subscription(
            create_subscription=ln.models.CreateSubscription(
                id=models.ResourceId(
                    scope=scope,
                    code=code
                ),
                display_name=name,
                description=description if description else name,
                status="Active",
                matching_pattern=ln.models.MatchingPattern(
                    event_type="FileCreated",
                    filter=matching_filter
                )
            )
        )
        print(f"Subscription created for {code}.")
        return resp
    except ln.ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 711: # 'SubscriptionAlreadyExists'
            raise e

# check if this already exists and delete if it does
subscriptions = subscriptions_api.list_subscriptions().values
for subscription in subscriptions:
    if subscription.id.code == "file_load" and subscription.id.scope == "file_load":
        subscriptions_api.delete_subscription(
            scope="file_load",
            code="file_load"
        )

create_file_subscription(
    scope="file_load",
    code="file_load",
    name="file_load",
    matching_filter="FileName eq 'auto-load-transactions.xlsx'",
)

Subscription created for file_load.


{'created_at': datetime.datetime(2023, 2, 7, 10, 12, 27, 756852, tzinfo=tzutc()),
 'created_by': '00uieujafoYdmkDSx2p7',
 'description': 'file_load',
 'display_name': 'file_load',
 'id': {'code': 'file_load', 'scope': 'file_load'},
 'last_modified_at': datetime.datetime(2023, 2, 7, 10, 12, 27, 756852, tzinfo=tzutc()),
 'last_modified_by': '00uieujafoYdmkDSx2p7',
 'matching_pattern': {'event_type': 'FileCreated',
                      'filter': "FileName eq 'auto-load-transactions.xlsx'"},
 'status': 'Active'}

### 3. Create a webhook notification

First, define the SQL Luminesce query to run. This query will unpack our data file from the drive and then create a portfolio, instruments and transactions.


In [6]:
sql = """
-- Extract transaction data from LUSID Drive

@txn_data =
use Drive.Excel
--file=/auto-load-transactions.xlsx
enduse;

-- Set variables for the portfolio's scope and code

@@portfolio_scope = select 'auto-load-demo';
@@portfolio_code = select 'uk-equity';
@@portfolio_name = select 'UK EQUITY';

-- Define the portfolio data

@create_portfolio =
select 'Transaction' as PortfolioType,
@@portfolio_scope as PortfolioScope,
@@portfolio_code as PortfolioCode,
@@portfolio_name as DisplayName,
'' as Description,
#2000-01-01# as Created,
''as SubHoldingKeys,
'GBP' as BaseCurrency;

-- Upload the portfolio into LUSID

@response_create_portfolio =
select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

-- Get instrument data

@equity_instruments =
select
Name as DisplayName,
ISIN as Isin,
ClientInternal as ClientInternal,
SEDOL as Sedol,
'GBP' as DomCcy
from @txn_data;

-- Upload the transformed data into LUSID

@response = select *
from Lusid.Instrument.Equity.Writer
where ToWrite = @equity_instruments;

-- --Transform data using SQL

@transactions =
select
@@portfolio_scope as PortfolioScope,
@@portfolio_code as PortfolioCode,
t.TransactionID as TxnId,
t.Type as Type,
t.TransactionDate as TransactionDate,
t.SettlementDate as SettlementDate,
t.Units as Units,
t.Price as TradePrice,
t.TotalConsideration as TotalConsideration,
t.Currency as SettlementCurrency,
t.ClientInternal as ClientInternal,
r.LusidInstrumentId as LusidInstrumentId
from @txn_data t
inner join @response r
where t.ClientInternal = r.ClientInternal;

-- Upload the transformed data into LUSID

select *
from Lusid.Portfolio.Txn.Writer
where ToWrite = @transactions;
"""

Now, define the webhook notification to run our SQL query.

In [7]:
def create_webhook_notification(
        scope: str,
        code: str,
        sql: str,
):
    # this can be changed to call any api required
    create_webhook_notification = {
        "description": "Upsert transaction into new portfolio",
        "httpMethod": "Put",
        "url": "/honeycomb/api/Sql/csv",
        "authenticationType": "Lusid",
        "contentType": "PlainText",
        "content": sql
    }

    try:
        return notifications_api.create_webhook_notification(
            scope=scope,
            code=code,
            create_webhook_notification=create_webhook_notification
        )
    except ln.ApiException as e:
        return e.body

# delete notifications to allow updates
notifications = notifications_api.list_notifications(scope="file_load", code="file_load").values
for notification in notifications:
    notifications_api.delete_notification(
        scope="file_load",
        code="file_load",
        id=notification.id
    )

create_webhook_notification(
    scope="file_load",  # scope and code of subscription
    code="file_load",
    sql=sql
)

{'content': {'Content': '\n'
                        '-- Extract transaction data from LUSID Drive\n'
                        '\n'
                        '@txn_data =\n'
                        'use Drive.Excel\n'
                        '--file=/auto-load-transactions.xlsx\n'
                        'enduse;\n'
                        '\n'
                        "-- Set variables for the portfolio's scope and code\n"
                        '\n'
                        "@@portfolio_scope = select 'auto-load-demo';\n"
                        "@@portfolio_code = select 'uk-equity';\n"
                        "@@portfolio_name = select 'UK EQUITY';\n"
                        '\n'
                        '-- Define the portfolio data\n'
                        '\n'
                        '@create_portfolio =\n'
                        "select 'Transaction' as PortfolioType,\n"
                        '@@portfolio_scope as PortfolioScope,\n'
                        '@@portfolio_code as 

### 4. Trigger the event

You can trigger the event by dropping the `auto-load-transactions.xlsx` file into drive. This file is located within the data folder. You can verify the process has been succesful by checking your portfolios for `uk-equity` within the `auto-load-demo` scope.